In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [4]:
def read_rdd(path):
    rdd = sc.textFile(path).map(lambda x: x.split('\t'))
    return rdd


def raiting_count(rdd, film=None):
    if film is None:
        dicts = sorted(rdd.map(lambda x : x[2], 1).countByKey().items())
    else:
        dicts = sorted(rdd.filter(lambda x: film == x[1]).map(lambda x : x[2], 1).countByKey().items())
    return dicts


def write_json(hist_film, hist_all):
    hist_film = []
    hist_all = []
    for k, v in raiting_count(rdd, '96'):
        hist_film.append(v)
    for k, v in raiting_count(rdd):
        hist_all.append(v) 
        
    resault = {'hist_film':hist_film, 'hist_all':hist_all}  
    
    with open("lab01.json", "w") as fp:
        json.dump(resault , fp) 

In [5]:
rdd = read_rdd('/labs/laba01/ml-100k/u.data')

hist_film = raiting_count(rdd, '96')
hist_all = raiting_count(rdd)
            
write_json(hist_film, hist_all)       

In [6]:
spark.stop()